In [145]:
import numpy as np
import pandas as pd

In [146]:
# Define the KNN class
class KNN:
    def __init__(self, k=9, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        # Store the training data
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def predict(self, X):
        predictions = []
        
        for x in X:
            # Compute distances from the test point to all training points
            distances = self.compute_distance(self.X_train, x)
            
            # Get the indices of the k nearest neighbors manually (without argsort)
            sorted_indices = self.manual_sort(distances)
            k_nearest_indices = sorted_indices[:self.k]
            
            # Get the labels of the k nearest neighbors
            k_nearest_labels = [self.y_train[i] for i in k_nearest_indices]
            
            # Majority vote for classification without Counter
            most_common_label = self.majority_vote(k_nearest_labels)
            
            predictions.append(most_common_label)
        
        return np.array(predictions)

    def manual_sort(self, distances):
        # Manual sorting to get indices of sorted distances
        return np.argsort(distances)  # Use argsort to avoid relying on Counter

    def majority_vote(self, labels):
        # Implement majority vote without Counter
        label_count = {}
        for label in labels:
            if label in label_count:
                label_count[label] += 1
            else:
                label_count[label] = 1
        # Get the label with the highest count
        max_label = None
        max_count = -1
        for label, count in label_count.items():
            if count > max_count:
                max_count = count
                max_label = label
        return max_label
    
    # Extend the compute_distance function to handle additional distance metrics
    def compute_distance(self, X_train, x_test):
        if self.distance_metric == 'euclidean':
            # Euclidean distance: sqrt(sum((x1 - x2)^2))
            distances = np.sqrt(np.sum((X_train - x_test) ** 2, axis=1))

        elif self.distance_metric == 'manhattan':
            # Manhattan distance: sum(abs(x1 - x2))
            distances = np.sum(np.abs(X_train - x_test), axis=1)

        elif self.distance_metric == 'chebyshev':
            # Chebyshev distance: max(|x1 - x2|)
            distances = np.max(np.abs(X_train - x_test), axis=1)

        elif self.distance_metric == 'minkowski':
            # Minkowski distance (generalized Euclidean and Manhattan): (sum(abs(x1 - x2)^p))^(1/p)
            p = 3  # Example value for p, can be parameterized
            distances = np.sum(np.abs(X_train - x_test) ** p, axis=1) ** (1 / p)

        elif self.distance_metric == 'cosine':
            # Cosine similarity-based distance: 1 - (dot(x1, x2) / (||x1|| * ||x2||))
            dot_product = np.sum(X_train * x_test, axis=1)
            X_train_norm = np.linalg.norm(X_train, axis=1)
            x_test_norm = np.linalg.norm(x_test)
            cosine_similarity = dot_product / (X_train_norm * x_test_norm)
            distances = 1 - cosine_similarity

        elif self.distance_metric == 'hamming':
            # Hamming distance: proportion of different elements between x1 and x2
            distances = np.mean(X_train != x_test, axis=1)

        elif self.distance_metric == 'mahalanobis':
            # Mahalanobis distance: sqrt((x1 - x2)^T * S^-1 * (x1 - x2)), where S is covariance matrix
            cov_matrix = np.cov(X_train.T)
            inv_cov_matrix = np.linalg.inv(cov_matrix)
            diff = X_train - x_test
            distances = np.sqrt(np.sum(np.dot(diff, inv_cov_matrix) * diff, axis=1))

        else:
            raise ValueError(f"Unknown distance metric: {self.distance_metric}")

        return distances




In [151]:
# Manually scale the features using basic mean-variance scaling (standardization)
def manual_min_max_scaler(X):
    # Compute min and max for each column
    X_min = np.min(X, axis=0)
    X_max = np.max(X, axis=0)

    # Apply min-max normalization
    return (X - X_min) / (X_max - X_min)

def preprocess_data(train_path, test_path):
    # Load the training and testing data
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Combine train and test for consistent preprocessing
    combined_data = pd.concat([train_data, test_data], axis=0)
    
    # Drop unnecessary columns
    combined_data.drop(['id','CustomerId', 'Surname'], axis=1, inplace=True)

    # Handle categorical variables manually (Geography and Gender)
    combined_data['Gender'] = combined_data['Gender'].apply(lambda x: 1 if x == 'Male' else 0)
    
    # Manually convert 'Geography' to one-hot encoding
    geography_unique = combined_data['Geography'].unique()
    for country in geography_unique:
        combined_data[f'Geography_{country}'] = (combined_data['Geography'] == country).astype(int)
    
    # Drop the original 'Geography' column
    combined_data.drop('Geography', axis=1, inplace=True)
    print(combined_data.columns)
    # Split the combined data back into train and test sets
    train_data = combined_data.iloc[:train_data.shape[0], :]
    test_data = combined_data.iloc[train_data.shape[0]:, :]
    
    # Separate features and target variable from training data
    X_train = train_data.drop('Exited', axis=1)
    y_train = train_data['Exited']

    # For the test set, remove the 'Exited' column if it exists
    if 'Exited' in test_data.columns:
        X_test = test_data.drop('Exited', axis=1)
    else:
        X_test = test_data
    
    # Normalize the features using Min-Max normalization
    X_train_scaled = manual_min_max_scaler(X_train.values)
    X_test_scaled = manual_min_max_scaler(X_test.values)

    return X_train_scaled, X_test_scaled, y_train

In [152]:
def cross_validate(X, y, knn, n_splits=5):
    # Combine X and y into a single array for easier splitting
    data = np.column_stack((X, y))
    
    # Shuffle the data manually
    np.random.seed(42)
    np.random.shuffle(data)
    
    # Split the data into 'n_splits' folds
    fold_size = len(data) // n_splits
    roc_auc_scores = []

    for i in range(n_splits):
        # Create validation fold
        val_data = data[i * fold_size:(i + 1) * fold_size]
        X_val = val_data[:, :-1]
        y_val = val_data[:, -1]
        
        # Remaining data for training
        train_data = np.concatenate([data[:i * fold_size], data[(i + 1) * fold_size:]], axis=0)
        X_train = train_data[:, :-1]
        y_train = train_data[:, -1]

        # Train the KNN classifier
        knn.fit(X_train, y_train)
        
        # Make predictions on the validation set
        y_val_pred = knn.predict(X_val)
        
        # Compute the ROC AUC score manually
        roc_auc = manual_roc_auc_score(y_val, y_val_pred)
        roc_auc_scores.append(roc_auc)
    
    # Return the average ROC AUC score across all folds
    return np.mean(roc_auc_scores)

def manual_roc_auc_score(y_true, y_pred):
    # Sort the true labels and predicted values by predicted scores
    sorted_indices = np.argsort(y_pred)
    y_true_sorted = y_true[sorted_indices]
    
    # Count positive and negative instances
    P = np.sum(y_true_sorted == 1)
    N = np.sum(y_true_sorted == 0)
    
    # Compute the rank sum for positive class instances
    rank_sum = np.sum(np.where(y_true_sorted == 1)[0] + 1)  # Rank starts from 1, hence +1

    # Compute the AUC using the rank-sum formula
    auc = (rank_sum - (P * (P + 1)) / 2) / (P * N)
    
    return auc


In [153]:
# Assume that preprocess_data, KNN, and cross_validate have already been defined.

# Load and preprocess data
X_train, X_test, y_train = preprocess_data('train.csv', 'test.csv')

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

print(f"Shape of X_train: {X_train[0]}")
print(f"Shape of X_train: {X_train[1]}")
print(f"Shape of X_train: {X_train[2]}")

# Hyperparameter tuning: Explore different values of k
best_k = None
best_score = 0
k_values = [3, 5, 7, 9, 11, 20, 30, 50]  # You can expand this range

for k in k_values:
    # Initialize KNN with Mahalanobis distance (this should be implemented in the KNN class)
    knn = KNN(k=k, distance_metric='mahalanobis')
    
    # Perform cross-validation to evaluate the current k
    cv_score = cross_validate(X_train, y_train, knn)
    
    print(f"K: {k}, Cross-validation ROC AUC Score: {cv_score}")
    
    # Check if this is the best k so far
    if cv_score > best_score:
        best_score = cv_score
        best_k = k

# Output the best hyperparameters
print(f"Best K: {best_k} with ROC AUC Score: {best_score}")

# Train on the full dataset using the optimal k value
knn = KNN(k=best_k, distance_metric='mahalanobis')
knn.fit(X_train, y_train)

# Make predictions on the test set
test_predictions = knn.predict(X_test)

# Save test predictions
# Assuming that the test set has an 'id' column in the CSV
test_ids = pd.read_csv('test.csv')['id']
submission = pd.DataFrame({'id': test_ids, 'Exited': test_predictions})
submission.to_csv('submissions.csv', index=False)

print("Predictions saved to submissions.csv")


Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_Germany', 'Geography_France', 'Geography_Spain'],
      dtype='object')
Shape of X_train: (15000, 12)
Shape of y_train: (15000,)
Shape of X_train: [0.57279236 1.         0.26785714 0.4        0.49603424 0.
 1.         1.         0.88238529 1.         0.         0.        ]
Shape of X_train: [0.55369928 1.         0.19642857 0.35       0.         0.33333333
 0.         0.         0.80905524 0.         1.         0.        ]
Shape of X_train: [0.66587112 1.         0.25       0.05       0.         0.33333333
 1.         0.         0.21996877 0.         1.         0.        ]
K: 3, Cross-validation ROC AUC Score: 0.7645501773841
K: 5, Cross-validation ROC AUC Score: 0.7643701185432726
K: 7, Cross-validation ROC AUC Score: 0.7623661073282313
K: 9, Cross-validation ROC AUC Score: 0.7652544222851777
K: 11, Cross-validation ROC AUC S